In [2]:
video_path = "/data1/hongn/chunks/video/usc_s1_1_0.avi"
audio_path = "/data1/hongn/chunks/audio/usc_s1_1_0.wav"

In [3]:
from transformers import AutoModel, Wav2Vec2Processor, VivitModel, VivitImageProcessor, VivitConfig, AutoImageProcessor, TimesformerModel
from tqdm import tqdm
import cv2
import argparse
import time
import numpy as np
import torch
import librosa

/data2/hongn/miniconda3/envs/sapiens_lite/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data2/hongn/miniconda3/envs/sapiens_lite/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch

# load model and processor
audio_processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
audio_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

In [5]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests
import torch.nn as nn


## get VIDEO EMBEDDINGS 

In [6]:
def get_frames(video_path):
    cap = cv2.VideoCapture(video_path)

    frames = []
    image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
    while True:
        ret, frame = cap.read()
        fps = cap.get(cv2.CAP_PROP_FPS)      # OpenCV v2.x used "CV_CAP_PROP_FPS"
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frame_count/fps
        if not ret:
            break
        
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (64, 64))
        frames.append(frame)
    cap.release()
    frames = np.array(frames, dtype=np.float32)  # Ensure float32 type
    # num_frames = len(frames)
    # print("num_frames original" , num_frames)
    return frames


In [7]:
frames.shape

NameError: name 'frames' is not defined

## get AUDIO EMBEDDINGS 

In [ ]:
def get_audio_target(audio_path, audio_model, audio_processor):
    # load dummy dataset and read soundfiles
    y, sr = librosa.load(audio_path, sr = 16000)
    rtmri, _ = librosa.effects.trim(y)

    # tokenize
    input_values = audio_processor(y, sampling_rate=16000, return_tensors="pt").input_values

    # retrieve logits
    with torch.no_grad():
        logits = audio_model(input_values).logits


    # take argmax and decode
    predicted_ids = torch.argmax(logits, dim=-1)

    return predicted_ids
    # transcription = processor.batch_decode(predicted_ids)
    # print(predicted_ids.shape)
    # transcription
    # # => should give ['m ɪ s t ɚ k w ɪ l t ɚ ɹ ɪ z ð ɪ ɐ p ɑː s əl ʌ v ð ə m ɪ d əl k l æ s ᵻ z æ n d w iː ɑːɹ ɡ l æ d t ə w ɛ l k ə m h ɪ z ɡ ɑː s p əl']
    # predicted_ids.shape


## get MODELs

In [8]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests
import torch.nn as nn

# url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.classifier = nn.Linear(in_features = 768, out_features = 392)
frames = get_frames(video_path)

# inputs = processor(images=frames[0], return_tensors="pt")
# outputs = model(**inputs)
# logits = outputs.logits
# # model predicts one of the 1000 ImageNet classes
# predicted_class_idx = logits.argmax(-1).item()
# print("Predicted class:", model.config.id2label[predicted_class_idx])


In [62]:
predicted_ids.shape[1]

198

In [68]:
inputs = processor(images=frames, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
# predicted_class_idx = logits.argmax(-1).item()
print(logits)

tensor([[-0.4506,  0.5200,  0.2072,  ...,  0.7388, -0.2510, -0.1924],
        [-0.5719,  0.6823,  0.3420,  ...,  0.5835, -0.4215, -0.1237],
        [-0.5401,  0.7185,  0.4396,  ...,  0.4447, -0.5279, -0.0339],
        ...,
        [-0.0658,  0.3333, -0.2515,  ...,  0.7546,  0.0931, -0.2396],
        [ 0.0427,  0.0400, -0.3734,  ...,  0.8814,  0.1702, -0.2340],
        [ 0.0634,  0.1863, -0.4782,  ...,  1.0285,  0.2061, -0.3605]],
       grad_fn=<AddmmBackward0>)


In [69]:
log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

In [81]:
predicted_ids.shape

torch.Size([1, 198])

In [86]:
predicted_ids.shape[0]

1

In [91]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, gamma=0.9, step_size=5)
best_loss = float('inf')

In [88]:
criterion = nn.CTCLoss(blank=0)
log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
loss = criterion(log_probs.unsqueeze(1), predicted_ids, input_lengths=tuple([1]), target_lengths=tuple([1]))

In [92]:
with torch.autograd.set_detect_anomaly(True):
    loss.backward()

In [ ]:
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
optimizer.step()
epoch_loss += loss.item()
num_batches += 1

In [8]:
import glob

video_paths = glob.glob("/data1/hongn/chunks/video/*.avi")
audio_paths = ["/data1/hongn/chunks/audio/" + i.split('/')[-1].split('.')[0] + ".wav" for i in video_paths]

In [11]:
all_spk = ['sub002',
'sub003',
'sub004',
'sub005',
'sub007',
'sub008',
'sub009',
'sub010',
'sub011',
'sub013',
'sub014',
'sub015',
'sub016',
'sub017',
'sub018',
'sub019',
'sub020',
'sub023',
'sub025',
'sub027',
'sub031',
'sub036',
'sub040',
'sub042',
'sub043',
'sub044',
'sub045',
'sub046',
'sub047',
'sub048',
'sub049',
'sub050',
'sub054',
'sub055',
'sub057',
'sub059',
'sub067',
'sub070',
'sub071',
'sub072']

video_paths = []
for sub in all_spk:
    video_paths = video_paths + glob.glob(f"/data1/span_data/rtmri75s/{sub}/2drt/video/*.mp4")
audio_paths = ["/data1/span_data/rtmri75s/" + i.split('/')[-4] + "/2drt/audio/" + i.split('/')[-1].split('.')[0][:-5] + "audio.wav" for i in video_paths]

In [10]:
video_paths

['/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_04_bvt_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_02_vcv2_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_05_shibboleth_r1_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_18_topic2_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_14_picture3_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_09_northwind1_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_05_shibboleth_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_01_vcv1_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_04_bvt_r1_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_10_northwind2_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_08_grandfather2_r1_video.mp4',
 '/data1/span_data/rtmri75s/sub002/2drt/video/sub002_2drt_06_rainbow_r2_video.m

In [7]:
import glob
video_paths = glob.glob("/data1/span_data/rtmri75s/sub00[2,3,4,5,7,8,9]/2drt/video/*.mp4")
audio_paths = ["/data1/span_data/rtmri75s/" + i.split('/')[-4] + "/2drt/audio/" + i.split('/')[-1].split('.')[0][:-5] + "audio.wav" for i in video_paths]
video_paths

['/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_01_vcv1_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_08_grandfather2_r1_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_13_picture2_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_03_vcv3_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_05_shibboleth_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_12_picture1_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_07_grandfather1_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_21_topic5_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_11_postures_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_10_northwind2_r2_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_15_picture4_video.mp4',
 '/data1/span_data/rtmri75s/sub007/2drt/video/sub007_2drt_06_rainbow_r1_vid

In [ ]:
import glob

# video_paths = glob.glob("/data1/hongn/chunks/video/*.avi")
# audio_paths = ["/data1/hongn/chunks/audio/" + i.split('/')[-1].split('.')[0] + ".wav" for i in video_paths]

video_paths = glob.glob("/data1/span_data/rtmri75s/sub*/2drt/video/*.mp4")
audio_paths = ["/data1/span_data/rtmri75s/" + i.split('/')[-4] + "/2drt/audio/" + i.split('/')[-1].split('.')[0][:-5] + "audio.wav" for i in video_paths]

FPS = 86
AUDIO_UNIT = 50

device = 'cuda:2'
image_processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.classifier = nn.Linear(in_features = 768, out_features = 392)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, gamma=0.9, step_size=5)
best_loss = float('inf')
criterion = nn.CTCLoss(blank=0)

EPOCH = 1
model = model.to(device)
for e in range(EPOCH):
    epoch_loss = 0
    for i in range(len(video_paths)):
        optimizer.zero_grad()
        video_path = video_paths[i]
        audio_path = audio_paths[i]
        frames = get_frames(video_path)
        input_len = int(len(frames)/86)
        print(frames.shape, input_len)
        # assert False
        # frames = frames[:input_len*99].to(device)
        # predicted_ids = get_audio_target(audio_path, audio_model, audio_processor).to(device)[:,:input_len*50].to(device)
        # print(frames.shape, input_len)
        # assert False
        for j in range(input_len):
            inputs = image_processor(images=frames[j*FPS:(j+1)*FPS], return_tensors="pt").to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
            predicted_ids = get_audio_target(audio_path, audio_model, audio_processor).to(device)[:,j*AUDIO_UNIT:(j+1)*AUDIO_UNIT]

            loss = criterion(log_probs.unsqueeze(1), predicted_ids, input_lengths=tuple([1]), target_lengths=tuple([1]))
            
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()
            # print(loss)
            epoch_loss += loss.item()

    print(f"epoch {e} loss: ", epoch_loss)

    if(epoch_loss < best_loss):
        best_loss = epoch_loss
        torch.save(model, "/data1/hongn/chunks/ckpt_pretrained_vits_on_span/best.pt")


(3566, 64, 64, 3) 41


AssertionError: 

In [15]:
video_path



'/data1/hongn/chunks/video/usc_s1_30_9.avi'